In [0]:
from pyspark.sql import SparkSession

# =====================================================
# 1. SESSÃO SPARK
# =====================================================
spark = SparkSession.builder.getOrCreate()

# =====================================================
# 2. CONFIGURAÇÕES
# =====================================================

# Origem (somente leitura)
catalogo_origem = "dataexperts"
schema_gold = "gold"

# Destino (novo, isolado)
catalogo_export = "dataexports"
schema_export = "export"
volume_export = "export_csv"

# Caminho físico final
base_destino = f"/Volumes/{catalogo_export}/{schema_export}/{volume_export}"

# =====================================================
# 3. CRIAÇÃO DO CATÁLOGO DE EXPORTAÇÃO
# =====================================================
spark.sql(f"""
CREATE CATALOG IF NOT EXISTS {catalogo_export}
""")

# =====================================================
# 4. CRIAÇÃO DO SCHEMA DE EXPORTAÇÃO
# =====================================================
spark.sql(f"""
CREATE SCHEMA IF NOT EXISTS {catalogo_export}.{schema_export}
""")

# =====================================================
# 5. CRIAÇÃO DO VOLUME PARA CSV
# =====================================================
spark.sql(f"""
CREATE VOLUME IF NOT EXISTS
{catalogo_export}.{schema_export}.{volume_export}
""")

print("📁 Catálogo, schema e volume de exportação prontos.")

# =====================================================
# 6. LISTAR TODAS AS TABELAS DA GOLD
# =====================================================
tabelas_gold = (
    spark.sql(f"SHOW TABLES IN {catalogo_origem}.{schema_gold}")
         .select("tableName")
         .collect()
)

if not tabelas_gold:
    raise Exception("❌ Nenhuma tabela encontrada na camada GOLD.")

print("📦 Tabelas encontradas na GOLD:")
for t in tabelas_gold:
    print(f" - {t.tableName}")

# =====================================================
# 7. EXPORTAR CADA TABELA PARA CSV
# =====================================================
for t in tabelas_gold:
    nome_tabela = t.tableName

    print(f"\n🚀 Exportando tabela: {nome_tabela}")

    # leitura da tabela gold
    df = spark.table(f"{catalogo_origem}.{schema_gold}.{nome_tabela}")

    # caminhos
    pasta_tabela = f"{base_destino}/{nome_tabela}"
    pasta_tmp = f"{pasta_tabela}/_tmp"

    # escrita temporária (Spark sempre cria pasta)
    (
        df.coalesce(1)
          .write
          .mode("overwrite")
          .option("header", "true")
          .csv(pasta_tmp)
    )

    # localizar o CSV gerado
    arquivos = dbutils.fs.ls(pasta_tmp)
    arquivo_csv = [f.path for f in arquivos if f.name.endswith(".csv")][0]

    # mover para nome final
    caminho_final = f"{pasta_tabela}/{nome_tabela}.csv"
    dbutils.fs.mkdirs(pasta_tabela)
    dbutils.fs.mv(arquivo_csv, caminho_final)

    # limpar pasta temporária
    dbutils.fs.rm(pasta_tmp, recurse=True)

    print(f"✅ {nome_tabela}.csv exportado com sucesso.")

# =====================================================
# 8. FINALIZAÇÃO
# =====================================================
print("\n🎉 EXPORTAÇÃO FINALIZADA COM SUCESSO!")
print(f"📂 Arquivos disponíveis em: {base_destino}")
